# `clip_retrieval.clip_client`

This python module allows you to query a backend remote via its exposed REST api.

## Install

In [1]:
%pip install clip-retrieval img2dataset

You should consider upgrading via the '/home/rom1504/clip-retrieval/.env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Setup

In [58]:
from IPython.display import Image, display
from clip_retrieval.clip_client import ClipClient, Modality

IMAGE_BASE_URL = "https://github.com/rom1504/clip-retrieval/raw/main/tests/test_clip_inference/test_images/"

def log_result(result):
    id, caption, url, similarity = result["id"], result["caption"], result["url"], result["similarity"]
    print(f"id: {id}")
    print(f"caption: {caption}")
    print(f"url: {url}")
    print(f"similarity: {similarity}")
    display(Image(url=url, unconfined=True))

client = ClipClient(
    url="https://knn.laion.ai/knn-service",
    indice_name="laion5B-L-14",
    aesthetic_score=9,
    aesthetic_weight=0.5,
    modality=Modality.IMAGE,
    num_images=10,
)


## Query by text

In [3]:
cat_results = client.query(text="an image of a cat")
log_result(cat_results[0])

id: 518836491
caption: orange cat with supicious look stock photo
url: https://media.istockphoto.com/photos/orange-cat-with-supicious-look-picture-id907595140?k=6&amp;m=907595140&amp;s=612x612&amp;w=0&amp;h=4CTvSxNvv4sxSCPxViryha4kAjuxDbrXM5vy4VPOuzk=
similarity: 0.5591723918914795


## Query by image

In [4]:
beach_results = client.query(image="https://github.com/rom1504/clip-retrieval/raw/main/tests/test_clip_inference/test_images/321_421.jpg")
log_result(beach_results[0])

id: 574870177
caption: Palm trees in Orlando, Florida
url: https://www.polefitfreedom.com/wp-content/uploads/2018/03/Orlando.jpg
similarity: 0.9619368314743042


## Query by embedding

In [67]:
import clip  # pylint: disable=import-outside-toplevel
import torch

model, preprocess = clip.load("ViT-L/14", device="cpu", jit=True)

import urllib
import io
import numpy as np
def download_image(url):
    urllib_request = urllib.request.Request(
        url,
        data=None,
        headers={"User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:72.0) Gecko/20100101 Firefox/72.0"},
    )
    with urllib.request.urlopen(urllib_request, timeout=10) as r:
        img_stream = io.BytesIO(r.read())
    return img_stream
def normalized(a, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2 == 0] = 1
    return a / np.expand_dims(l2, axis)

def get_text_emb(text):
    with torch.no_grad():
        text_emb = model.encode_text(clip.tokenize([text], truncate=True).to("cpu"))
        text_emb /= text_emb.norm(dim=-1, keepdim=True)
        text_emb = text_emb.cpu().detach().numpy().astype("float32")[0]
    return text_emb

from PIL import Image as pimage

def get_image_emb(image_url):
    with torch.no_grad():
        image = pimage.open(download_image(image_url))
        image_emb = model.encode_image(preprocess(image).unsqueeze(0).to("cpu"))
        image_emb /= image_emb.norm(dim=-1, keepdim=True)
        image_emb = image_emb.cpu().detach().numpy().astype("float32")[0]
        return image_emb


In [68]:
red_tshirt_text_emb =  get_text_emb("red tshirt")
red_tshirt_results = client.query(embedding_input=red_tshirt_text_emb.tolist())
log_result(red_tshirt_results[0])

id: 3142652315
caption: KCTS29 100% Cotton Unisex Kids T Shirt Crew Neck V Neck Solid Maroon
url: https://cdn.shopify.com/s/files/1/1531/9423/products/CTS29_100_Cotton_T_Shirt_Crew_Neck_V_Neck_Long_Sleeves_Solid_Maroon_27f4c580-c286-4e20-b7d7-110c4b2a3ff3_large.jpg?v=1476875623
similarity: 0.5375761389732361


In [69]:
blue_dress_image_emb = get_image_emb("https://rukminim1.flixcart.com/image/612/612/kv8fbm80/dress/b/5/n/xs-b165-royal-blue-babiva-fashion-original-imag86psku5pbx2g.jpeg?q=70")
blue_dress_results = client.query(embedding_input=blue_dress_image_emb.tolist())
log_result(blue_dress_results[0])

id: 2463946620
caption: 8c7889e0b92b Cinderella Divine 1295 Long Chiffon Grecian Royal Blue Dress Mid Length  Sleeves V Neck ...
url: https://cdn.shopify.com/s/files/1/1417/0920/products/1295cd-royal-blue_cfcbd4bc-ed74-47c0-8659-c1b8691990df.jpg?v=1527650905
similarity: 0.9430060386657715


In [72]:
red_tshirt_text_emb =  get_text_emb("red tshirt")
blue_dress_image_emb = get_image_emb("https://rukminim1.flixcart.com/image/612/612/kv8fbm80/dress/b/5/n/xs-b165-royal-blue-babiva-fashion-original-imag86psku5pbx2g.jpeg?q=70")
mean_emb = normalized(red_tshirt_text_emb + blue_dress_image_emb)[0]
mean_results = client.query(embedding_input=mean_emb.tolist())
log_result(mean_results[0])

id: 2702080924
caption: CLEARANCE - Long Chiffon Grecian Red Dress Mid Length Sleeves V Neck (Size Medium)
url: https://cdn-img-3.wanelo.com/p/716/c27/0c0/aef7a32a4317370b6f7f14b/x354-q80.jpg
similarity: 0.824600338935852


## Download and format a dataset from the results of a query

If you have some images of your own, you can query each one and collect the results into a custom dataset (a small subset of LAION-5B)

In [4]:
# Create urls from known images in repo
import json
from tqdm import tqdm
test_images = [f"{IMAGE_BASE_URL}{image}" for image in ["123_456.jpg", "208_495.jpg", "321_421.jpg", "389_535.jpg", "416_264.jpg", "456_123.jpg", "524_316.jpg"]]

# Re-initialize client with higher num_images
client = ClipClient(url="https://knn.laion.ai/knn-service", indice_name="laion5B-L-14", num_images=40)

# Run one query per image
combined_results = []
for image in tqdm(test_images):
    combined_results.extend(client.query(image=image))

# Save results to json file
with open("search-results.json", "w") as f:
    json.dump(combined_results, f)

100%|██████████| 7/7 [00:17<00:00,  2.44s/it]


In [5]:
!img2dataset "search-results.json" --input_format="json" --caption_col "caption" --output_folder="laion-enhanced-dataset" --resize_mode="no" --output_format="files"

Starting the downloading of this file
Sharding file number 1 of 1 called /home/samsepiol/Projects/clip-retrieval/notebook/search-results.json
0it [00:00, ?it/s]File sharded in 1 shards
1it [00:32, 32.08s/it]
worker  - success: 0.921 - failed to download: 0.079 - failed to resize: 0.000 - images per sec: 2 - count: 76
total   - success: 0.921 - failed to download: 0.079 - failed to resize: 0.000 - images per sec: 2 - count: 76


In [6]:
import os
print(f"Done! Download/copy the contents of {os.getcwd()}/laion-enhanced-dataset/")
!realpath laion-enhanced-dataset/

Done! Download/copy the contents of /home/samsepiol/Projects/clip-retrieval/notebook/laion-enhanced-images/
/home/samsepiol/Projects/clip-retrieval/notebook/laion-enhanced-images/00000.tar
